In [1]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

### 1. Policy update after every action
    - Mutation of policy, which is the correct can only break the correct setup
    - `compare_last_reward__no_memory__random`, `compare_last_reward__with_memory`

In [2]:
policy_history = None
t = Tasks.CupToDrawer()
for i in range(1,101):
    # create start and target scenes for this task
    s, s2 = t.get_start_target_scenes()
    # state includes action names, gesture names ...
    state = s.generate_scene_state(Actions.A, Gestures.G, Users.U, Users.selected_id)
    # mapping bayes network create
    bn = MappingBayesNet(state, policy_history=policy_history)
    bn.policy_history
    # Creates diagonal policy as prior
    bn.init_policy_simple()

    reward = 0.
    done = False
    n=0
    # not reaching the target in <100 steps, it breaks
    while not done and n<100:
        # t.decicde simulates user behavior and chooses an action&object which user wants to perform
        # based on it, observation is created, demonstrates the gesture performed by the user
        bn.create_observation(t.decide(s))
        # this gesture and object info is taken and the predicted action is computed
        action = bn.select_action()
        # evaluates the given action and computes a reward
        s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
        # here the policy is updated with every action
        bn.policy_update(reward, out=False, mode='compare_last_reward__no_memory__random')
        if not step_done: reward = -999
        n+=1
    print(f"n: {n}")
    policy_history = deepcopy(bn.policy_history)
    #print(bn.policy)

n: 100
n: 10
n: 100
n: 18
n: 100
n: 10
n: 100
n: 29
n: 22
n: 31
n: 5
n: 100
n: 100
n: 26
n: 34
n: 45
n: 100
n: 26
n: 3
n: 6
n: 3
n: 100
n: 100
n: 3
n: 100
n: 100
n: 100
n: 33
n: 11
n: 100
n: 19
n: 100
n: 100
n: 3
n: 11
n: 3
n: 100
n: 100
n: 100
n: 6
n: 12
n: 100
n: 100
n: 100
n: 3
n: 25
n: 100
n: 3
n: 100
n: 4
n: 3
n: 100
n: 3
n: 100
n: 6
n: 15
n: 6
n: 100
n: 10
n: 3
n: 19
n: 27
n: 17
n: 3
n: 3
n: 5
n: 10
n: 26
n: 17
n: 100
n: 100
n: 100
n: 25
n: 3
n: 100
n: 3
n: 100
n: 4
n: 100
n: 100
n: 3
n: 100
n: 23
n: 3
n: 3
n: 100
n: 31
n: 3
n: 9
n: 3
n: 100
n: 3
n: 7
n: 3
n: 100
n: 100
n: 3
n: 3
n: 8
n: 100


In [3]:
for i in range(719):
    bn.policy_update(reward, out=False, mode='compare_last_reward__by_selection')

In [4]:
def get_best_reward_policy(policy_history):
    rewards = []
    for policy in policy_history:
        rewards.append(policy['r'])

    for i in range(5):
        winner = np.argmax(rewards)
        print(f"{winner}: {rewards[winner]}, {policy_history[winner]['CM_est']}")

        rewards.pop(winner)
        policy_history.pop(winner)

### 2. Policy update after every action: Improvement
`compare_last_reward__by_selection`
- Saves used permutations of policy


In [5]:
policy_history = None
t = Tasks.CupToDrawer()
all_visited = False
for i in range(1,101):
    # create start and target scenes for this task
    s, s2 = t.get_start_target_scenes()
    # state includes action names, gesture names ...
    state = s.generate_scene_state(Actions.A, Gestures.G, Users.U, Users.selected_id)
    # mapping bayes network create
    bn = MappingBayesNet(state, policy_history=policy_history)
    bn.policy_history
    # Creates diagonal policy as prior
    bn.init_policy_simple()

    reward = 0.
    done = False
    n=0
    # not reaching the target in <100 steps, it breaks
    while not done and n<100:
        # t.decicde simulates user behavior and chooses an action&object which user wants to perform
        # based on it, observation is created, demonstrates the gesture performed by the user
        bn.create_observation(t.decide(s))
        # this gesture and object info is taken and the predicted action is computed
        action = bn.select_action()
        # evaluates the given action and computes a reward
        s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
        # here the policy is updated with every action
        if bn.policy_update(reward, out=False, mode='compare_last_reward__by_selection') is False:
            print("All permutations visited")
            all_visited = True
            break
        if not step_done: reward = -999
        n+=1
    if all_visited: break
    print(f"n: {n}")
    policy_history = deepcopy(bn.policy_history)
    #print(bn.policy)

n: 29
n: 13
n: 17
n: 16
n: 9
n: 13
n: 10
n: 41
n: 32
n: 10
n: 64
n: 10
n: 8
n: 10
n: 28
n: 10
n: 10
n: 20
n: 55
n: 48
n: 11
n: 8
n: 21
n: 42
n: 6
n: 8
n: 13
n: 11
n: 14
n: 36
n: 10
n: 28
n: 28
n: 22
n: 5
n: 39
n: 10
n: 7
n: 22
All permutations visited


In [6]:
policy_history.pop(-1)

False

In [7]:
get_best_reward_policy(policy_history)

1: 5.0, [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]]
1: 5.0, [[0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]]
1: 5.0, [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]]
1: 5.0, [[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
1: 5.0, [[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]]


### Encode scene state to int number

In [1]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [2]:
s = Scene(init='cup,cup,drawer')

In [3]:
s.scene_encode_to_state()

96

### 3. Policy update every done task
- Reward is computed as number of steps of the task from start to the end

### Policy update by Q-Learning

In [1]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [2]:
alpha = 0.1
gamma = 0.6
epsilon = 0.1

In [3]:
policy_history = [
{'CM_est': np.array([[0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'init',
 'r': True,
 'permutation': 0,
 'action': {'target_object': 'drawer', 'target_action': 'put'}}
]
t = Tasks.CupToDrawer()
all_visited = False
for i in range(1,101):
    # create start and target scenes for this task
    s, s2 = t.get_start_target_scenes()
    # state includes action names, gesture names ...
    state = s.generate_scene_state(Actions.A, Gestures.G, Users.U, Users.selected_id)
    # mapping bayes network create
    bn = MappingBayesNet(state, policy_history=policy_history)
    # Creates diagonal policy as prior
    #bn.init_policy_simple()
    
    reward = 0.
    done = False
    n=0
    # not reaching the target in <100 steps, it breaks
    while not done and n<100:
        # t.decicde simulates user behavior and chooses an action&object which user wants to perform
        # based on it, observation is created, demonstrates the gesture performed by the user
        bn.create_observation(t.decide(s))
        # this gesture and object info is taken and the predicted action is computed
        action = bn.select_action()
        # evaluates the given action and computes a reward
        s, reward, done, step_done = Actions.step(s, s2, action, ignore_location=True, out=False)
        # here the policy is updated with every action
        policy = PolicyUpdateQLearning.do(bn.policy_history, reward, action, bn.observation, alpha, gamma)
        bn.policy_history.append(deepcopy(policy))
        if policy is False or policy is None:
            print("All permutations visited")
            all_visited = True
            break
        if not step_done: reward = -999
        n+=1
    if all_visited: break
    print(f"n: {n}")
    policy_history = deepcopy(bn.policy_history)
    #print(bn.policy)

=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 

=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
n: 100
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1

=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 

=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 

=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 

=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
n: 100
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1

=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
n: 100
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1 0, 1: 1::: 1
=== 1

KeyboardInterrupt: 

In [17]:
bn.policy

{'CM_est': array([[0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'init',
 'r': 4.0,
 'permutation': 0,
 'action': {'target_object': 'drawer', 'target_action': 'move_up'}}

In [18]:
bn.policy_history[-10:]

[{'CM_est': array([[0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]),
  'i': 'init',
  'r': 4.0,
  'permutation': 0,
  'action': {'target_object': 'drawer', 'target_action': 'move_up'}},
 {'CM_est': array([[0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]),
  'i': 'init',
  'r': 4.0,
  'permutation': 0,
  'action': {'target_object': 'drawer', 'target_action': 'move_up'}},
 {'CM_est': array([[0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]),
  'i': 'init',
  'r': 4.0,
  'permutation': 0,
  'action': {'target_object': 'drawer', 'target_action': 'move_up'}},
 {'CM_est': array([[0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0